In [ ]:
import pandas as pd
import itertools as it
import string
from collections import Counter
import json
import re
import numpy
import spacy

In [ ]:
with open('data/stopwords.txt') as fh:
    stopwords = set(fh.read().splitlines())

In [ ]:
spacy_nlp = spacy.load("en_core_web_lg")

In [ ]:
def normalise(text):
    
    text = text.lower()
    text = text.replace("\n", " ")
    text = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " <url> ", text)
    text = text.replace("/", " / ")
    text = re.sub(r"@\w+", " <user> ", text)
    text = re.sub(r"[8:=;]['`\-][)d]+|[)d]+['`\-][8:=;]", " <smile> ", text)
    text = re.sub(r"[8:=;]['`\-]p+", " <lolface> ", text)
    text = re.sub(r"[8:=;]['`\-]\(+|\)+['`\-][8:=;]", " <sadface> ", text)
    text = re.sub(r"[8:=;]['`\-][\/|l*]", " <neutralface> ", text)
    text = text.replace(r"<3", " <heart> ")
    text = re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " <number> ", text)
    text = text.replace(r"#", " <hashtag> ")
    text = re.sub(r"([!?.,()])+", r" \1 ", text)
    
    return text

In [ ]:
def tokenise(text):
    return [t.strip() for t in text.strip().split()]

In [ ]:
def parse_treatment_definitons(definion_file):
    treatment_set = set()
    treatment_mapping = {}
    max_length = 1
    for line in definion_file:
        line = line.strip()
        treatments = line.split(',')
        name = treatments[0]
        for treatment in treatments:
            treatment = tuple(tokenise(normalise(treatment)))
            max_length = max(len(treatment), max_length)
            treatment_set.add(treatment)
            treatment_mapping[treatment] = name
    return treatment_set, treatment_mapping, max_length

In [ ]:
def window_sliding(iterable, n):
    gens = (
        it.chain(it.repeat(None, n - 1 - i), iterable, it.repeat(None, i))
        for i, gen in enumerate(it.tee(iterable, n)))
    return zip(*gens)

def find_treatments(tokens):
    found_treatments = []
    for x in range(max_treatment_length, 0, -1):
        for window in window_sliding(tokens, x):
            if tuple(window) in treatment_set:
                found_treatments.append(treatment_mapping[tuple(window)])
    return list(set(found_treatments)) or None

In [ ]:
def filter_stopwords(tokens):
    return [t for t in tokens if len(t) > 2 and t not in stopwords]

In [ ]:
def calculate_embeddings(tokens):
    token_embdgs = [embeddings.get(t) for t in tokens]
    token_embdgs = [t for t in token_embdgs if t is not None]
    return numpy.mean(token_embdgs, axis=0)

In [ ]:
def add_pos_tags(df):
    tokens = []
    pos = []
    tag = []

    for doc in spacy_nlp.pipe(df['text'].astype('unicode').values, batch_size=50, n_threads=3):
        if doc.is_parsed:
            tokens.append([n.text for n in doc])
            pos.append([n.pos_ for n in doc])
            tag.append([n.tag_ for n in doc])
        else:
            tokens.append(None)
            pos.append(None)
            tag.append(None)

    df['tokens'] = tokens
    df['pos'] = pos
    df['tag'] = tag
    return df

In [ ]:
with open('data/tweets.jsonl', 'r') as fh:
    df = pd.read_json(fh.read(), lines=True, convert_dates=True)

In [ ]:
df = df.loc[(df['retweeted_status'].isnull()) & (df['lang'] == "en")]

In [ ]:
df = df.loc[(df.created_at >= "2017-12-04") & (df.created_at <= "2018-02-03")]

In [ ]:
df["extended_tweet"] = df["extended_tweet"].apply(lambda x: x["full_text"]  if type(x) == dict else x )

In [ ]:
df["text"] = df["extended_tweet"].combine_first(df["text"])

In [ ]:
df["text"] = df["text"].apply(lambda x: normalise(x).replace("\n", " "))

In [ ]:
df["user_name"] = df["user"].apply(lambda x: x["screen_name"])

In [ ]:
df = df.loc[:, ["id", "created_at", "text", "retweet_count", "user_name"]]

In [ ]:
df = add_pos_tags(df)

In [ ]:
noun_tags = {"NN", "NNS", "NNP", "NNPS"}
adj_tags = {"JJ", "JJR", "JJS"}
nouns = Counter()
adjectives = Counter()
for _, row in df.iterrows():
    tokens = row["tokens"]
    tags = row["tag"]
    for cnt, tag in enumerate(tags):
        if tag in noun_tags:
            nouns[tokens[cnt]] += 1
        if tag in adj_tags:
            adjectives[tokens[cnt]] += 1

In [ ]:
nouns.most_common()

In [ ]:
adjectives.most_common()

In [ ]:
with open("data/nouns_twitter.json", 'w') as fh:
    json.dump(nouns, fh)
with open("data/adjectives_twitter.json", 'w') as fh:
    json.dump(adjectives, fh)

In [ ]:
df["tokens"] = df["text"].apply(lambda x: tokenise(normalise(x)))

In [ ]:
len(df.user_name.unique())

In [ ]:
treatment_set, treatment_mapping, max_treatment_length = parse_treatment_definitons(open("data/treatment_definitons.txt", 'r'))

In [ ]:
df['filtered_tokens'] = df['tokens'].apply(filter_stopwords)

In [ ]:
word_counts = Counter()
for elem in df["filtered_tokens"]:
    word_counts += Counter(elem)

In [ ]:
word_counts.most_common(1)

In [ ]:
df['treatments'] = df['tokens'].apply(find_treatments)

In [ ]:
df = df.loc[df['treatments'].notnull()]

In [ ]:
len(df.user_name.unique())

In [ ]:
df.to_csv("tweets_with_treatments.csv")

In [ ]:
treatment_counts = Counter()
for elem in df["treatments"]:
    treatment_counts += Counter(elem)

In [ ]:
new_df_twitter=pd.DataFrame(treatment_counts.most_common(100))

In [ ]:
new_df_twitter

In [ ]:
new_df_twitter.columns= ['keyword','count']

In [ ]:
new_df_twitter

In [ ]:
new_df_twitter.info()


In [ ]:
new_df_twitter.to_pickle('Twitter_counts.pkl') 

In [ ]:
new_df_TT = pd.read_pickle('TT_counts.pkl')

In [ ]:
new_df_TT.columns= ['keyword','count']
new_df_TT

In [ ]:
pd.concat([new_df_TT, new_df_twitter], axis=1)

In [ ]:
new_df_all= pd.merge(new_df_TT, new_df_twitter, how='outer', on= 'keyword')


In [ ]:
#get the numbers to distinct mentions of substances in TT and twitter
#NAN indicates no mention, x=TT , y=Twitter


new_df_all[new_df_all.isnull().any(axis=1)]
new_df_all.to_csv("forums_difference.csv")

In [ ]:
new_df_common=pd.merge(new_df_TT, new_df_twitter, how='inner', on= 'keyword')
new_df_common
TT_count_common = sum(new_df_common['count_x'])
Twitter_count_common=sum(new_df_common['count_y'])

In [ ]:
TT_count_common


In [ ]:
new_df_common

In [ ]:
#get relative counts with respect to the total mentions in each of the forum

#added  * 100 to  see if any difference in the stats score
#however no diffrence ( *100 can also be removed)
new_df_common['new_TT'] = new_df_common['count_x']*100 /  new_df_common['count_x'].sum()
new_df_common['new_Tw'] = new_df_common['count_y'] *100/  new_df_common['count_y'].sum()

In [ ]:
new_df_common


In [ ]:
new_df_common['count_x'].sum()

In [ ]:
new_df_common


In [ ]:
from scipy.stats import ttest_ind, ttest_ind_from_stats
t, p = ttest_ind(new_df_common['new_TT'], new_df_common['new_Tw'])
print(t)
print(p)

In [ ]:
from scipy.stats import ttest_ind
ttest_ind(new_df_common['new_TT'].values,new_df_common['new_Tw'].values)

In [ ]:
new_df_common['new_TT'].values

In [ ]:
type(new_df_common['new_TT'])